In [84]:
import inspect
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from numpy.linalg import norm
from scipy.optimize import line_search, brent
from functools import partial
from copy import deepcopy
from IPython.display import display, HTML
from scipy.optimize import line_search, fmin


def get_output(func, x_new, dataset_rec, flag,  dataset=None):
    """
    Generates output data for functions.
    """
    res = {'point': None, 'value_func': None, 'report': None,
           'interim_results_dataset': None}
    args_func = inspect.getfullargspec(func)[0]
    shape_arg = len(args_func)
    col = [f'x_{i}' for i in range(1, shape_arg+1)]
    res['point'] = x_new
    res['value_func'] = func(*x_new)
    if dataset_rec:
        res['interim_results_dataset'] = pd.DataFrame(dataset,
                                                      columns=col+['f'])
    res['report'] = flag
    return res


def _find_lr_rate(lr, func, diff_func, x):
    """
    Returns the value of the learning rate for the fastest descent.
    Used for GDSteepest.
    """
    return func(*[x[i] - lr*diff_func(*x)[i] for i in range(len(x))])


def GD(func, diff_func, lr_rate=0.1, x_old=None, accuracy=10**-5, maxiter=500,
       interim_results=False, dataset_rec=False):
    """
    Returns dict with the minimum of a function using the gradient descent,
    value of function, report and intermediate results in pandas.DataFrame
    (opt). Given the function and the derivative of the function, return the
    minimum of the function with the specified accuracy.
    Parameters
    ----------
    func : callable ``f(x)``
        Objective function to be minimized.
    diff_func : callable ``f'(x)``
        Derivative of the objective function.
    lr_rate : float
        Learning rate or step.
    x_old: list
        Starting point
    accuracy : float, optional
        stop criterion.
    maxiter : int
        Maximum number of iterations to perform.
    interim_results : bool, optional
        If True, print intermediate results.
    dataset_rec : bool, optional
        If True, an entry in pandas.DataFrame intermediate results.
    Examples
    --------
    >>> from HW3.gradient import *
    >>> def f(x, y):
            return x**2 + 2*y
    >>> def pf(x, y):
            return np.array([2*x, 2])
    >>> minimum = GD(f, pf, lr_rate=0.05)
    >>> print(minimum['point'])
    [1.3220708194808046e-23, -49.000000000000426]
    """
    flag = None
    dataset = []
    iterat = 0
    args_func = inspect.getfullargspec(func)[0]
    shape_arg = len(args_func)
    col = [f'x_{i}' for i in range(1, shape_arg+1)]
    if x_old is None:
        x_old = [1] * shape_arg
    crit_stop = accuracy*2
    while crit_stop > accuracy and iterat < maxiter:
        x_new = [x_old[i] - lr_rate * diff_func(*x_old)[i]
                 for i in range(shape_arg)]
        crit_stop = norm([x_new[i] - x_old[i] for i in range(shape_arg)])
        x_old = x_new
        if dataset_rec:
            dataset.append([*x_new, func(*x_new)])
        if interim_results:
            print(f'''{iterat}:
            point = {x_new}    f(point) = {func(*x_new)}''')
        iterat += 1
    if crit_stop <= accuracy and flag != 2:
        flag = 0
    elif iterat == maxiter and flag != 2:
        flag = 1
    return get_output(func, x_new, dataset_rec, flag, dataset)


In [12]:
res = GD(lambda x,y: -20*np.exp(-0.2*(0.5*x**2+y**2)**0.5)-np.exp(0.5*(np.cos(2*np.pi*x) + np.cos(2*np.pi*y))) + np.exp(1) + 20, 
    lambda x,y: np.array([2.0 * x * (0.5 * x ** 2 + y ** 2) ** (-0.5) * np.exp(
                -0.2 * (0.5 * x ** 2 + y ** 2) ** 0.5) + 3.14159265358979 * np.exp(
                0.5 * np.cos(6.28318530717959 * x) + 0.5 * np.cos(6.28318530717959 * y)) * np.sin(6.28318530717959 * x),
                4.0 * y * (0.5 * x ** 2 + y ** 2) ** (-0.5) * np.exp(
                -0.2 * (0.5 * x ** 2 + y ** 2) ** 0.5) + 3.14159265358979 * np.exp(
                0.5 * np.cos(6.28318530717959 * x) + 0.5 * np.cos(6.28318530717959 * y)) * np.sin(
                6.28318530717959 * y)]),
       lr_rate=0.1, x_old=None, accuracy=10**-5, maxiter=500,
       interim_results=False, dataset_rec=True)



In [13]:
dataset_rec = res['interim_results_dataset']

In [76]:
x = [i for i in dataset_rec['x_1'].tolist()]
y = [i for i in dataset_rec['x_2'].tolist()]

In [77]:
x_min, x_max = min(x), max(x)
y_min, y_max = min(y), max(y)

In [78]:
X = np.linspace(x_min, x_max, 100)
Y = np.linspace(y_min, y_max, 100)

In [79]:
X, Y = np.meshgrid(X, Y)

In [80]:
f = lambda x,y: -20*np.exp(-0.2*(0.5*x**2+y**2)**0.5) - \
    np.exp(0.5*(np.cos(2*np.pi*x) + np.cos(2*np.pi*y))) + np.exp(1) + 20


In [81]:
Z = f(X, Y)

In [87]:
scatter = go.Scatter3d(x=x, y=y, z=h['f'], marker={'size':3})

surface = go.Surface(z=Z, x=X, y=Y, colorscale='RdBu', opacity=0.3,  showscale=True)

In [88]:
fig = go.Figure(data=[scatter, surface])
fig.show()